In [16]:
import os
import requests
from pprint import pprint
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [ ]:
url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
params = {
    "_type": "json",
    'serviceKey' : os.getenv("API_KEY"), 
    'pageNo' : '1', 
    'numOfRows' : '10', 
    'solYear' : '2025', 
    'solMonth' : '06' 
}

response = requests.get(url, params=params)
pprint(response.json())

In [ ]:
for i in range(1, 13):
    params["solMonth"] = str(i).rjust(2, "0")
    response = requests.get(url, params=params)
    pprint(response.json())

In [ ]:
datetime.now().month

In [74]:
def get_holiday_in_months(month_range: int = 6) -> list[str]:
    result = []
    
    __now = datetime.now()
    __url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
    __params = {
        "_type": "json",
        'serviceKey' : os.getenv("API_KEY"), 
        'pageNo' : '1', 
        'numOfRows' : '20', 
        'solYear' : __now.year, 
        'solMonth' : f"{__now.month}".rjust(2, "0") 
    }
    
    for month in range(month_range + 1):
        __target_datetime = __now + relativedelta(months=month)
        __params["solYear"] = __target_datetime.year
        __params["solMonth"] = f"{__target_datetime.month}".rjust(2, "0")
        
        __resp = requests.get(__url, params=__params)
        
        if __resp.status_code == 200:
            # 결과가 없는 경우 str
            items: dict | str = __resp.json()["response"]["body"]["items"]
            
            if type(items) == dict:
                # 1개만 나오면 딕셔너리 아이템 하나만 있다.
                # list[dict]는 1개 이상일 떄 반환
                items: dict | list = items["item"]
                
                if type(items) == dict:
                    result.append(items)
                else:
                    result.extend(items)
    
    result = sorted(set([str(item["locdate"]) for item in result]))
    
    return result

In [ ]:
def get_weekdays_in_months(month_range: int = 6, date_format: str = "%Y%m%d") -> list[str]:
    result = []
    
    __now = datetime.now()
    __saterday = __now + timedelta(days=5-__now.weekday())
    __sunday = __now + timedelta(days=6-__now.weekday())   
    
    __last_month_day = (__now + relativedelta(months=month_range+1)).replace(day=1, hour=0, minute=0, second=0, microsecond=0)
    
    while __saterday < __last_month_day and __sunday < __last_month_day:
        if __saterday < __last_month_day:
            result.append(__saterday.strftime(date_format))
        if __sunday < __last_month_day:
            result.append(__sunday.strftime(date_format))
            
        __saterday += timedelta(days=7)
        __sunday += timedelta(days=7)
        
    return result
    
    

In [82]:
get_holiday_in_months(month_range=1)

['20250505', '20250506']

In [83]:
get_weekdays_in_months(month_range=1)

['20250412',
 '20250413',
 '20250419',
 '20250420',
 '20250426',
 '20250427',
 '20250503',
 '20250504',
 '20250510',
 '20250511',
 '20250517',
 '20250518',
 '20250524',
 '20250525']

In [34]:
def get_business_days_in_holidays(month_range: int = 6, holidays: list[str] = [], date_format: str = "%Y%m%d") -> dict[str, bool]:
    result = {}
        
    __now = datetime.now()
    __last_month_day = (__now + relativedelta(months=month_range+1)).replace(day=1, hour=0, minute=0, second=0, microsecond=0)

    for __day in range((__last_month_day - __now).days + 1):
        __date = (__now + timedelta(days=__day)).strftime(date_format)
        result[__date] = __date not in holidays
        
        
    return result    

In [ ]:
get_korean_business_days(
    month_range=1,
    holidays=[
        '20250412',
        '20250413',
        '20250419',
        '20250420',
        '20250426',
        '20250427',
        '20250503',
        '20250504',
        '20250505',
        '20250506',
        '20250510',
        '20250511',
        '20250517',
        '20250518',
        '20250524',
        '20250525'
    ]
)

{'20250408': True,
 '20250409': True,
 '20250410': True,
 '20250411': True,
 '20250412': False,
 '20250413': False,
 '20250414': True,
 '20250415': True,
 '20250416': True,
 '20250417': True,
 '20250418': True,
 '20250419': False,
 '20250420': False,
 '20250421': True,
 '20250422': True,
 '20250423': True,
 '20250424': True,
 '20250425': True,
 '20250426': False,
 '20250427': False,
 '20250428': True,
 '20250429': True,
 '20250430': True,
 '20250501': True,
 '20250502': True,
 '20250503': False,
 '20250504': False,
 '20250505': False,
 '20250506': False,
 '20250507': True,
 '20250508': True,
 '20250509': True,
 '20250510': False,
 '20250511': False,
 '20250512': True,
 '20250513': True,
 '20250514': True,
 '20250515': True,
 '20250516': True,
 '20250517': False,
 '20250518': False,
 '20250519': True,
 '20250520': True,
 '20250521': True,
 '20250522': True,
 '20250523': True,
 '20250524': False,
 '20250525': False,
 '20250526': True,
 '20250527': True,
 '20250528': True,
 '20250529': Tr